In [2]:
import numpy as np
import lime
import lime.lime_tabular
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Wczytanie danych
serce = pd.read_csv('serce.csv') 

# Zdefiniowanie zmiennych X i y
X = serce[['wiek', 'plec', 'typ_bolu_klatka', 'cisnienie_krwi_spoczynek', 
            'cholesterol_we_krwi', 'cukier_we_krwi', 
            'wynik_ekg_spoczynek', 'ilosc_uderzen_serca', 
            'bol_klatka_wysilek', 'max_obnizka_st', 
            'przebieg_st_szczyt', 'zwapnienia_miazdzycowe', 
            'proba_ta']].values
y = serce['diagnoza'].values

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tworzenie modelu RandomForest
model = RandomForestClassifier(n_estimators=50)
model.fit(X_train, y_train)

# Tworzenie wyjaśniacza LIME
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train, 
    feature_names=['wiek', 'plec', 'typ_bolu_klatka', 'cisnienie_krwi_spoczynek',  # przekazanie nazw cech
                   'cholesterol_we_krwi', 'cukier_we_krwi', 
                   'wynik_ekg_spoczynek', 'ilosc_uderzen_serca', 
                   'bol_klatka_wysilek', 'max_obnizka_st', 
                   'przebieg_st_szczyt', 'zwapnienia_miazdzycowe', 
                   'proba_ta'],
    class_names=['brak', 'choroba'],  # przekazanie nazw decyzji
    mode='classification'
)

# Wyjaśnienia dla jednej z instancji w zbiorze testowym
instance = X_test[1]    

explanation = explainer.explain_instance(
    data_row=instance,                  # przekazuje wybraną instancje do wyjaśnienia
    predict_fn=model.predict_proba,     # zwraca prawdopodobieństwo klas
    top_labels=1,                       # wyjaśnia klasę o najwyższym prawdopodobieństwie
    num_features=5,                     #  liczba cech uwzględnionych w wyjaśnieniu
    labels=[0]  
)

predicted_class = model.predict(instance.reshape(1, -1))[0]

print(f'Instancja dla klasy: {predicted_class}') #
print('Cechy:', instance)                        # Te linie odpowiadają za wyświetlenie cech 
print('Klasa przewidziana:', predicted_class)    # oraz prognozowanej klasy 
print('Wyjaśnienie LIME:')                       # 
explanation.show_in_notebook(show_all=False)


instance = X_test[15]                            

explanation = explainer.explain_instance(
    data_row=instance,
    predict_fn=model.predict_proba,
    top_labels=1,
    num_features=13, # W tym przypadku uwzględniane są wszystkie cechy
    labels=[2]
)

predicted_class = model.predict(instance.reshape(1, -1))[0]

print(f'Instancja dla klasy: {predicted_class}') #
print('Cechy:', instance)                        #
print('Klasa przewidziana:', predicted_class)    # 
print('Wyjaśnienie LIME:')                       #
explanation.show_in_notebook(show_all=False)



Instancja dla klasy: 2
Cechy: [ 46.    1.    4.  120.  249.    0.    2.  144.    0.    0.8   1.    0.
   7. ]
Klasa przewidziana: 2
Wyjaśnienie LIME:


Instancja dla klasy: 1
Cechy: [ 48.   1.   3. 124. 255.   1.   0. 175.   0.   0.   1.   2.   3.]
Klasa przewidziana: 1
Wyjaśnienie LIME:
